In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv('Social_Network_Ads.csv')

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [5]:
dataset=dataset.drop("User ID",axis=1)

In [6]:
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [7]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [8]:
dataset.columns

Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [9]:
independent=dataset[[ 'Age', 'EstimatedSalary','Gender_Male']]
dependent=dataset[['Purchased']]

In [10]:
from sklearn.model_selection  import train_test_split
X_train,X_test,y_train,y_test=train_test_split(independent,dependent,test_size=0.30,random_state=0)

In [11]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test) 

In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

param_grid={'criterion':['gini', 'entropy','log_loss'],'max_features':['sqrt','log2',None],'n_estimators':[50,100]}
grid=GridSearchCV( RandomForestClassifier(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid.fit(X_train,y_train)


Fitting 5 folds for each of 18 candidates, totalling 90 fits


C:\Anaconda\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_features': ['sqrt', 'log2', None],
                         'n_estimators': [50, 100]},
             scoring='f1_weighted', verbose=3)

In [13]:
re=grid.cv_results_

grid_predictions = grid.predict(X_test) 

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)

from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)


In [14]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)


The f1_macro value for best parameter {'criterion': 'entropy', 'max_features': 'log2', 'n_estimators': 100}: 0.9166666666666666


In [15]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[74  5]
 [ 5 36]]


In [16]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.94      0.94      0.94        79
           1       0.88      0.88      0.88        41

    accuracy                           0.92       120
   macro avg       0.91      0.91      0.91       120
weighted avg       0.92      0.92      0.92       120



In [17]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])


0.9560049397962334

In [18]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.177368,0.014926,0.023844,0.006807,gini,sqrt,50,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.874254,0.840114,0.859435,0.892857,0.963889,0.886110,0.042574,4
1,0.343401,0.017265,0.028082,0.004926,gini,sqrt,100,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.874254,0.875644,0.841398,0.911105,0.946153,0.889711,0.035817,3
2,0.188826,0.014791,0.020017,0.007063,gini,log2,50,"{'criterion': 'gini', 'max_features': 'log2', ...",0.874254,0.821429,0.841398,0.892857,0.909115,0.867810,0.032323,16
3,0.358613,0.033319,0.026250,0.001906,gini,log2,100,"{'criterion': 'gini', 'max_features': 'log2', ...",0.874254,0.838326,0.823129,0.911105,0.909115,0.871186,0.035863,13
4,0.210750,0.015254,0.022957,0.002425,gini,None,50,"{'criterion': 'gini', 'max_features': None, 'n...",0.855314,0.857143,0.824293,0.893717,0.927778,0.871649,0.035657,12
5,0.369325,0.017466,0.026918,0.002210,gini,None,100,"{'criterion': 'gini', 'max_features': None, 'n...",0.855314,0.875644,0.841398,0.875548,0.927778,0.875136,0.029335,11
6,0.170158,0.007273,0.016608,0.005574,entropy,sqrt,50,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.892857,0.857143,0.823129,0.911105,0.909115,0.878670,0.033848,10
7,0.336171,0.014742,0.022473,0.005949,entropy,sqrt,100,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.874254,0.857143,0.841398,0.911105,0.945469,0.885874,0.037754,6
8,0.167794,0.003657,0.021116,0.006321,entropy,log2,50,"{'criterion': 'entropy', 'max_features': 'log2...",0.874254,0.858503,0.859435,0.911105,0.945469,0.889753,0.033753,2
9,0.333313,0.005965,0.028042,0.004278,entropy,log2,100,"{'criterion': 'entropy', 'max_features': 'log2...",0.874254,0.857143,0.859435,0.911105,0.963889,0.893165,0.040291,1
